# Домашнее задание к уроку 1: Основы PyTorch

In [234]:
import torch

## Задание 1: Создание и манипуляции с тензорами

### 1.1 Создание тензоров

In [235]:
# Тензор размером 3x4, заполненный случайными числами от 0 до 1
tensor_3x4 = torch.rand(3, 4)
# Тензор размером 2x3x4, заполненный нулями
tensor_2x3x4 = torch.zeros(2, 3, 4)
# Тензор размером 5x5, заполненный единицами
tensor_5x5 = torch.ones(5, 5)
# Тензор размером 4x4 с числами от 0 до 15 (используйте reshape)
tensor_4x4 = torch.arange(16).reshape(4, 4)

### 1.2 Операции с тензорами

In [236]:
a = torch.rand(3, 4)
b = torch.rand(4, 3)

# Транспонирование тензора A
a_transposed = a.T
# Матричное умножение A и B
matrix_product = a @ b
# Поэлементное умножение A и транспонированного B
element_wise_product = a * b.T
# Вычислите сумму всех элементов тензора A
a_sum = a.sum()

### 1.3 Индексация и срезы

In [237]:
tensor = torch.rand(5, 5, 5)

# Первая строка
first_row = tensor[0, 0, :]
# Последний столбец
last_column = tensor[:, :, -1:]
# Подматрица размером 2x2 из центра тензора
center_2x2 = tensor[0, 2:4, 2:4]
# Все элементы с четными индексами
even_indices = tensor[::2, ::2, ::2]

### 1.4 Работа с формами

In [238]:
tensor = torch.arange(24)

# 2x12
tensor_2x12 = tensor.view(2, 12)
# 3x8
tensor_3x8 = tensor.view(3, 8)
# 4x6
tensor_4x6 = tensor.view(4, 6)
# 2x3x4
tensor_2x3x4 = tensor.view(2, 3, 4)
# 2x2x2x3
tensor_2x2x2x3 = tensor.view(2, 2, 2, 3)

## Задание 2: Автоматическое дифференцирование

### 2.1 Простые вычисления с градиентами

In [239]:
x = torch.tensor(1.0, requires_grad=True)
y_pred = torch.tensor(2.0, requires_grad=True)
z = torch.tensor(3.0, requires_grad=True)

f = x**2 + y_pred**2 + z**2 + 2 * x * y_pred * z
f.backward()

print("df/dx: ", x.grad)
print("df/dy: ", y_pred.grad)
print("df/dz: ", z.grad)

df/dx:  tensor(14.)
df/dy:  tensor(10.)
df/dz:  tensor(10.)


Проверка аналитически:

- df/dx = 2x + 2yz => 2 * 1 + 2 * 2 * 3 = 14
- df/dy = 2y + 2xz => 2 * 2 + 2 * 1 * 3 = 10
- df/dz = 2z + 2xy => 2 * 3 + 2 * 1 * 2 = 10

Все ответы совпали

### 2.2 Градиент функции потерь

In [240]:
def mse(
    x: torch.Tensor, w: torch.Tensor, b: torch.Tensor, y_true: torch.Tensor
) -> torch.Tensor:
    """Mean Squared Error

    Args:
        x (torch.Tensor): Входные данные.
        w (torch.Tensor): Веса модели.
        b (torch.Tensor): Смещение модели.
        y_true (torch.Tensor): Правильные ответы.

    Returns:
        torch.Tensor: Средняя квадратичная ошибка между предсказанными и правильными ответами.
    """
    y_pred = x @ w + b
    return ((y_pred - y_true) ** 2).mean()

Примеры

In [241]:
# Обычный пример использования функции mse
x = torch.arange(1, 5, dtype=torch.float32).reshape(2, 2)
w = torch.tensor([0.5] * 2, requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)
y_true = torch.tensor([4.0, 1.0])

f = mse(x, w, b, y_true)
f.backward()

print("df/dw: ", w.grad)
print("df/db: ", b.grad)

df/dw:  tensor([ 9., 11.])
df/db:  tensor([2.])


---

In [242]:
# Пример использования функции mse, когда предсказание == правильному ответу
x = torch.arange(30, dtype=torch.float32).view(10, 3)
w = torch.rand(3, 10, requires_grad=True)
b = torch.ones(10, requires_grad=True)
y_true = x @ w + b

f = mse(x, w, b, y_true)
f.backward()

# Модель дала идеальное предсказание => градиенты и функция потерь - нулевые
assert (f == 0).all()
assert (w.grad == 0).all()
assert (b.grad == 0).all()

---

In [243]:
# Пример с простыми числами
x = torch.tensor([2.0])
w = torch.tensor([1.0], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
y_true = torch.tensor([5.0])

f = mse(x, w, b, y_true)
f.backward()

print("df/dw: ", w.grad)
print("df/db: ", b.grad)

df/dw:  tensor([-12.])
df/db:  tensor([-6.])


Проверка аналитически:

f = (wx + b - y_true)^2

- df/dw = 2(wx + b - y_true) * x => 2 (1 * 2 - 5) * 2 = -12
- df/db = 2(wx + b - y_true) * 1 => 2 (1 * 2 - 5) = -6

Ответы совпали

### 2.3 Цепное правило

In [244]:
x = torch.tensor([2.0], requires_grad=True)
f = torch.sin(x**2 + 1)
f.backward()
print(f"df/dx (через .backward()): {x.grad.item()}")  # type: ignore

df/dx (через .backward()): 1.1346487998962402


Проверка аналитически:

f = sin(x^2 + 1)

df/dx = cos(x^2 + 1) * 2x => cos(4 + 1) * 4 => cos(5) * 4 = 1.13465

Проверка с помощью torch.autograd.grad:

In [245]:
x = torch.tensor([2.0], requires_grad=True)
f = torch.sin(x**2 + 1)
df_dx = torch.autograd.grad(f, x)[0]
print(f"df/dx (через autograd.grad): {df_dx.item()}")  # type: ignore

df/dx (через autograd.grad): 1.1346487998962402


## Задание 3: Сравнение производительности CPU vs CUDA

### 3.1 Подготовка данных 

In [246]:
# Создание больших матриц
a_cpu = torch.rand(64, 1024, 1024)
b_cpu = torch.rand(128, 512, 512)
c_cpu = torch.rand(256, 256, 256)

a_cuda = a_cpu.to("cuda")
b_cuda = b_cpu.to("cuda")
c_cuda = c_cpu.to("cuda")

### 3.2 Функция измерения времени

In [247]:
import time
from enum import Enum


class Device(Enum):
    CPU = "cpu"
    CUDA = "cuda"


def measure_time(
    func, *args, device: Device = Device.CPU, repeat: int = 10, **kwargs
) -> float:
    """Среднее время выполнения функции на заданном устройстве.

    Args:
        func (func): Функция, время выполнения которой нужно измерить.
        *args: Позиционные аргументы для функции.
        device (Device, optional): Устройство. Defaults to Device.CPU.
        repeat (int, optional): Количество повторений. Defaults to 10.
        **kwargs: Именованные аргументы для функции.

    Raises:
        RuntimeError: Если CUDA не доступен, а устройство указано как Device.CUDA.
        ValueError: Если устройство указано неверно.

    Returns:
        float: Среднее время выполнения функции в миллисекундах.
    """
    times = []
    for _ in range(repeat):
        if device == Device.CPU:
            start = time.time()
            func(*args, **kwargs)
            end = time.time()
            times.append((end - start) * 1000)
        elif device == Device.CUDA:
            if not torch.cuda.is_available():
                raise RuntimeError("CUDA не доступен")

            start_event = torch.cuda.Event(enable_timing=True)
            end_event = torch.cuda.Event(enable_timing=True)

            torch.cuda.synchronize()
            start_event.record()  # type: ignore

            func(*args, **kwargs)

            end_event.record()  # type: ignore
            torch.cuda.synchronize()
            times.append(start_event.elapsed_time(end_event))
        else:
            raise ValueError("Неверное устройство, должно быть cpu или cuda")
    return sum(times) / len(times)

### 3.3 Сравнение операций

In [248]:
from dataclasses import dataclass, field


# Класс для хранения результатов измерения времени выполнения операций
@dataclass
class OperationResult:
    operation_name: str
    _results: list[tuple[Device, float, torch.Size]] = field(default_factory=list)

    def add_result(self, device: Device, time_ms: float, matrix_size: torch.Size):
        """Добавляет результат измерения времени выполнения операции.

        Args:
            device (Device): Устройство, на котором измерялось время.
            time_ms (float): Время выполнения операции в миллисекундах.
            matrix_size (torch.Size): Размер матрицы, на которой выполнялась операция.
        """
        self._results.append((device, time_ms, matrix_size))

    def get_result(
        self, device: Device, matrix_size: torch.Size | None = None
    ) -> float:
        """Получает среднее время выполнения операции на заданном устройстве.

        Args:
            device (Device): Устройство, для которого нужно получить результат.
            matrix_size (torch.Size | None, optional): Размер матрицы, для которой нужно получить результат
            (Если None, то берутся все размеры). Defaults to None.

        Returns:
            float: Среднее время выполнения операции в миллисекундах.
        """
        result = [
            res
            for res in self._results
            if res[0] == device and (matrix_size is None or res[2] == matrix_size)
        ]
        return sum(res[1] for res in result) / len(result) if result else 0.0

    def get_speedup(self, matrix_size: torch.Size | None = None) -> float:
        """Получает ускорение операции на GPU относительно CPU.

        Args:
            matrix_size (torch.Size | None, optional): Размер матрицы, для которой нужно получить результат
            (Если None, то берутся все размеры). Defaults to None.

        Returns:
            float: Ускорение операции на GPU относительно CPU.
        """
        return (
            self.get_result(Device.CPU, matrix_size)
            / self.get_result(Device.CUDA, matrix_size)
            if self.get_result(Device.CUDA, matrix_size) > 0
            else float("inf")
        )

In [249]:
# Список для хранения результатов измерения времени выполнения операций
operations: list[OperationResult] = []

Измерение матричного умножения

In [250]:
operation = OperationResult("Матричное умножение")
operations.append(operation)
for cpu_tensor in (a_cpu, b_cpu, c_cpu):
    operation.add_result(
        Device.CPU,
        measure_time(torch.matmul, cpu_tensor, cpu_tensor, device=Device.CPU),
        cpu_tensor.shape,
    )
for cuda_tensor in (a_cuda, b_cuda, c_cuda):
    operation.add_result(
        Device.CUDA,
        measure_time(torch.matmul, cuda_tensor, cuda_tensor, device=Device.CUDA),
        cuda_tensor.shape,
    )

Измерение поэлементного сложения

In [251]:
operation = OperationResult("Поэлементное сложение")
operations.append(operation)
for cpu_tensor in (a_cpu, b_cpu, c_cpu):
    operation.add_result(
        Device.CPU,
        measure_time(torch.add, cpu_tensor, cpu_tensor, device=Device.CPU, repeat=100),
        cpu_tensor.shape,
    )
for cuda_tensor in (a_cuda, b_cuda, c_cuda):
    operation.add_result(
        Device.CUDA,
        measure_time(
            torch.add, cuda_tensor, cuda_tensor, device=Device.CUDA, repeat=100
        ),
        cuda_tensor.shape,
    )

Измерение поэлементного умножения

In [252]:
operation = OperationResult("Поэлементное умножение")
operations.append(operation)
for cpu_tensor in (a_cpu, b_cpu, c_cpu):
    operation.add_result(
        Device.CPU,
        measure_time(torch.mul, cpu_tensor, cpu_tensor, device=Device.CPU, repeat=100),
        cpu_tensor.shape,
    )
for cuda_tensor in (a_cuda, b_cuda, c_cuda):
    operation.add_result(
        Device.CUDA,
        measure_time(
            torch.mul, cuda_tensor, cuda_tensor, device=Device.CUDA, repeat=100
        ),
        cuda_tensor.shape,
    )

Измерение транспонирования

In [253]:
operation = OperationResult("Транспонирование (transpose_copy)")
operations.append(operation)
for cpu_tensor in (a_cpu, b_cpu, c_cpu):
    operation.add_result(
        Device.CPU,
        measure_time(
            torch.transpose_copy, cpu_tensor, -1, -2, device=Device.CPU, repeat=100
        ),
        cpu_tensor.shape,
    )
for cuda_tensor in (a_cuda, b_cuda, c_cuda):
    operation.add_result(
        Device.CUDA,
        measure_time(
            torch.transpose_copy, cuda_tensor, -1, -2, device=Device.CUDA, repeat=100
        ),
        cuda_tensor.shape,
    )

Измерение вычисления суммы всех элементов

In [254]:
operation = OperationResult("Сумма всех элементов")
operations.append(operation)
for cpu_tensor in (a_cpu, b_cpu, c_cpu):
    operation.add_result(
        Device.CPU,
        measure_time(torch.sum, cpu_tensor, device=Device.CPU, repeat=1000),
        cpu_tensor.shape,
    )
for cuda_tensor in (a_cuda, b_cuda, c_cuda):
    operation.add_result(
        Device.CUDA,
        measure_time(torch.sum, cuda_tensor, device=Device.CUDA, repeat=1000),
        cuda_tensor.shape,
    )

#### Результаты

Общие результаты

In [259]:
from prettytable import PrettyTable


print("ОБЩИЕ РЕЗУЛЬТАТЫ ИЗМЕРЕНИЯ ВРЕМЕНИ ВЫПОЛНЕНИЯ ОПЕРАЦИЙ")
table = PrettyTable()
table.field_names = ["Операция", "CPU (мс)", "GPU (мс)", "Ускорение"]
for operation in operations:
    table.add_row(
        [
            operation.operation_name,
            f"{operation.get_result(Device.CPU):.2f}",
            f"{operation.get_result(Device.CUDA):.2f}",
            f"{operation.get_speedup():.2f}",
        ]
    )
print(table)

ОБЩИЕ РЕЗУЛЬТАТЫ ИЗМЕРЕНИЯ ВРЕМЕНИ ВЫПОЛНЕНИЯ ОПЕРАЦИЙ
+-----------------------------------+----------+----------+-----------+
|              Операция             | CPU (мс) | GPU (мс) | Ускорение |
+-----------------------------------+----------+----------+-----------+
|        Матричное умножение        |  784.54  |  39.92   |   19.66   |
|       Поэлементное сложение       |  43.48   |   3.56   |   12.21   |
|       Поэлементное умножение      |  44.37   |   3.93   |   11.29   |
| Транспонирование (transpose_copy) |  117.85  |   4.51   |   26.14   |
|        Сумма всех элементов       |   6.37   |   1.61   |    3.97   |
+-----------------------------------+----------+----------+-----------+


Результаты по тензорам

In [261]:
from prettytable import PrettyTable


for tensor in (a_cpu, b_cpu, c_cpu):
    print()
    print("ТЕНЗОР: ", " x ".join(map(str, tensor.shape)))
    table = PrettyTable()
    table.field_names = ["Операция", "CPU (мс)", "GPU (мс)", "Ускорение"]
    for operation in operations:
        table.add_row(
            [
                operation.operation_name,
                f"{operation.get_result(Device.CPU, tensor.shape):.2f}",
                f"{operation.get_result(Device.CUDA, tensor.shape):.2f}",
                f"{operation.get_speedup(tensor.shape):.2f}",
            ]
        )
    print(table)
    print()


ТЕНЗОР:  64 x 1024 x 1024
+-----------------------------------+----------+----------+-----------+
|              Операция             | CPU (мс) | GPU (мс) | Ускорение |
+-----------------------------------+----------+----------+-----------+
|        Матричное умножение        | 1804.03  |  96.00   |   18.79   |
|       Поэлементное сложение       |  75.28   |   6.73   |   11.19   |
|       Поэлементное умножение      |  80.18   |   7.90   |   10.15   |
| Транспонирование (transpose_copy) |  262.76  |   8.96   |   29.33   |
|        Сумма всех элементов       |  10.93   |   2.67   |    4.09   |
+-----------------------------------+----------+----------+-----------+


ТЕНЗОР:  128 x 512 x 512
+-----------------------------------+----------+----------+-----------+
|              Операция             | CPU (мс) | GPU (мс) | Ускорение |
+-----------------------------------+----------+----------+-----------+
|        Матричное умножение        |  424.47  |  18.72   |   22.67   |
|       По

### 3.4 Анализ результатов


#### Какие операции получают наибольшее ускорение на GPU?

Лидеры по ускорению:
- ```transpose_copy``` - до x29 ускорения на тензоре 64 x 1024 x 1024
- ```matmul``` - до x25 ускорения на тензоре 256 x 256 x 256
- Поэлементные операции (сложение, умножение) - стабильное ускорение x10–14

#### Почему некоторые операции медленнее на GPU или ускоряются слабее?

Меньше всех ускоряется операция ```sum``` (примерно в 4 раза). Так происходит потому что эта операция - операция сведения и ее невозможно эффективно распараллелить без синхронизации: каждая часть суммы вычисляется в отдельном потоке, а затем все частичные результаты необходимо объединить, что требует координации и обмена данными между потоками.

Также время выполнения операции ```sum``` самое низкое (около 1-2 мс), и в таких условиях накладные расходы на запуск CUDA ядра, выделение памяти и синхронизацию становятся значительными по сравнению с полезной работой. В итоге, даже если сама операция выполняется быстрее на GPU, общая стоимость вычислений (включая подготовку и завершение) снижает эффективность и уменьшает итоговое ускорение.

#### Как размер матриц влияет на ускорение?

Проанализируем ускорение для самой маленькой и самой большой матрицы:
| Операция | Большая матрица | Маленькая матрица | Прирост ускорения маленькой матрицы |
| ---------| --------------- | ----------------- | ----------------------------------- |
| Матричное умножение | 18.79 | 24.89 | +32% |
| Поэлементное сложение | 11.19 | 13.14 | +17% |
| Поэлементное умножение | 10.15 | 13.20 | +30% |
| Транспонирование | 29.33 | 15.70 | -47% |
| Сумма всех элементов | 4.09 | 3.53 | -14% |

GPU обычно выигрывает на больших объемах данных, где он может раскрыть весь потенциал параллелизма. Но почему в моем случае в 3 из 5 случаев ускорение маленькой матрицы больше. Скорее всего это связано с тем что у CPU производительность быстрее падает на маленьких матрицах: CPU - это несколько мощных ядер, и он неэффективен на мелких задачах с высокой частотой обращений к памяти. То есть маленькие матрицы сильно просаживают CPU, но в то же время могут достаточно хорошо загрузить GPU. 

Также важно учитывать, что в таблице показано относительное ускорение, а не абсолютная производительность. GPU может работать примерно одинаково быстро на любых матрицах, а CPU может быть на больших матрицах медленнее незначительно.

#### Что происходит при передаче данных между CPU и GPU?

При передаче данных между CPU и GPU происходит копирование тензора из оперативной памяти (RAM) в видеопамять (VRAM) через шину PCI Express, что является относительно медленной и синхронной операцией.